Vocabulary size = 30K

In [45]:
from BERTQG.token_generation import load_model, generate_token

class QuestionGeneration():
    def __init__(self, lm_gq, uqa_qg, regularization):
        self.lm_qg, _, _ = load_model(lm_qg)
        self.uqa_qg, self.tokenizer, device = load_model(uqa_qg)
        self.regularization = regularization
        
    def generate_token(self, context: str, ans: str, ans_start: int, list_question_tokens: list) -> str:
        '''
        Input:
            - context
            - ans
            - ans_start
            - list_question_tokens: the history of generated question tokens.
            The form of list_question_tokens is a list of (q_token, class), where class is 0 for lm
            token and 1 for uqa token
        Output:
            - The next generated question token
        '''
        # contains the tokens of the generated question
        list_question_tokens = []
        # contains the classes of each token of the gen. question. Same len as list_question_tokens
        # 0 = LM, 1 = UQA
        list_token_classes = []
        qi = None
        
        while True:
            # Generate the toknes and probs of the ith query token using the lm and uqa models
            question_text = " ".jon(list_question_tokens)
            lm_qi_token, lm_qi_idx, lm_qi_probs = generate_token(lm_qg, tokenizer, device, context, question_text, ans, ans_start)
            uqa_qi_token, uqa_qi_idx, uqa_qi_probs = generate_token(uqa_qg, tokenizer, device, context, question_text, ans, ans_start)
            
            # Get the final token  prob distribution using regulatization
            qi_probs, qi_class = self.regularization(lm_qi_probs, uqa_qi_probs, 
                                                     list_question_tokens, list_token_classes)

            # Get the predicted token
            if qi_class == 0: # LM
                qi = lm_qi_token
            else: # UQA
                qi = uqa_qi_token
                
            if self.finished_generation(qi): # [SEP]
                break
            list_question_tokens.append(qi)
            list_token_classes.append(qi_class)

        return " ".join(list_question_tokens)
            
    def finished_generation(self, question_token):
        return question_token =='[SEP]'
    
    def __convert2squad(self, context: str, answer: str, ans_start: int,  question: str) -> dict:
        '''
        Create a SQuAD instance
        Inputs:
            - context: paragrah
            - answer
            - ans_start
            - question: might not be the full question (we are generating questions token by token)
        Returns:
            - squad instance
        '''
        squad = {'data': [], 'version': '1.0'}
        squad['data'].append({'paragraphs': [{'title': 'title', 
                                              'context': context,
                                              'qas': [{'answers': [{'answer_start': ans_start, 'text': answer}],
                                                       'question': question,
                                                       'id': 0}]}]})
        return squad

In [1]:
import json
with open('../datasets/squad_NER_ans_full_wo_questions_evenly_distributed.json', 'r') as f:
    squad = json.load(f)

In [8]:
squad['data'][0]['paragraphs'][0]

{'title': 'University_of_Notre_Dame',
 'context': 'Architecturally , the school has a Catholic character . Atop the Main Building \'s gold dome is a golden statue of the Virgin Mary . Immediately in front of the Main Building and facing it , is a copper statue of Christ with arms upraised with the legend " Venite Ad Me Omnes " . Next to the Main Building is the Basilica of the Sacred Heart . Immediately behind the basilica is the Grotto , a Marian place of prayer and reflection . It is a replica of the grotto at Lourdes , France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858 . At the end of the main drive ( and in a direct line that connects through 3 statues and the Gold Dome ) , is a simple , modern stone statue of Mary .',
 'qas': [{'answers': [{'answer_start': 476, 'text': 'France'}],
   'question': '',
   'id': 0}]}

# Testing

## Vocabulary

In [9]:
import urllib.request
import numpy as np
word_url = "http://svnweb.freebsd.org/csrg/share/dict/words?view=co&content-type=text/plain"
response = urllib.request.urlopen(word_url)
long_txt = response.read().decode()
VOCAB_IDX2_STR = np.array(long_txt.splitlines()).reshape(-1,1)

In [10]:
VOCAB_IDX2_STR.shape

(25487, 1)

In [23]:
import torch
import torch.nn as nn

In [24]:
softmax = nn.Softmax(dim=0)

In [29]:
qi_probs = softmax(torch.empty(VOCAB_IDX2_STR.shape[0]).normal_(mean=4,std=0.5))

In [30]:
qi = torch.argmax(qi_probs, dim=0)

In [33]:
VOCAB_IDX2_STR[qi].item()

'Valparaiso'